In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv')
df.head()

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature
0,2013-11-18,90,月,0,厚切りイカフライ,NaN,NaN,NaN,NaN,快晴,--,19.8
1,2013-11-19,101,火,1,手作りヒレカツ,NaN,NaN,NaN,NaN,快晴,--,17.0
2,2013-11-20,118,水,0,白身魚唐揚げ野菜あん,NaN,NaN,NaN,NaN,快晴,--,15.5
3,2013-11-21,120,木,1,若鶏ピリ辛焼,NaN,NaN,NaN,NaN,快晴,--,15.2
4,2013-11-22,130,金,1,ビッグメンチカツ,NaN,NaN,NaN,NaN,快晴,--,16.1


## week×soldout毎のyの平均値を算出する

In [3]:
g_df = df.groupby(['week', 'name'])[['name']].count().rename(columns={'name':'count'})
display(g_df.head(), g_df.tail())

count
week name               
月    きのこソースハンバーグ       1
     カキフライタルタル         1
     サーモンフライ・タルタル      1
     ジューシーメンチカツ        1
     タンドリーチキン          1

count
week name                  
金    酢豚orカレー              1
     鶏のレモンペッパー焼orカレー      1
     鶏の味噌漬け焼き             1
     鶏の唐揚げ甘酢あん            1
     鶏の天ぷら                1

In [4]:
reset_g_df = df.groupby(['week', 'name'])[['name']].count().rename(columns={'name':'count'}).reset_index()
display(reset_g_df.head(), reset_g_df.tail())

,week,name,count
0,月,きのこソースハンバーグ,1
1,月,カキフライタルタル,1
2,月,サーモンフライ・タルタル,1
3,月,ジューシーメンチカツ,1
4,月,タンドリーチキン,1


,week,name,count
181,金,酢豚orカレー,1
182,金,鶏のレモンペッパー焼orカレー,1
183,金,鶏の味噌漬け焼き,1
184,金,鶏の唐揚げ甘酢あん,1
185,金,鶏の天ぷら,1


## BoW(メニュー名出現回数)

In [5]:
pivot_df = pd.pivot_table(reset_g_df, index='week', columns='name', values='count', aggfunc='sum', fill_value=0)
pivot_df

name,いか天ぷら,かじきの甘辛煮,きのこソースハンバーグ,さっくりメンチカツ,さわら焼味噌掛け,さんま辛味焼,たっぷりベーコンフライ,ひやしたぬきうどん・炊き込みご飯,ぶりレモンペッパー焼き,ますのマスタードソース,...,鶏の唐揚げ甘酢あん,鶏の塩から揚げ,鶏の天ぷら,鶏の照り焼きマスタード,鶏の照り焼きマヨ,鶏の親子煮,鶏チリソース,鶏肉とカシューナッツ炒め,鶏肉のカレー唐揚,鶏肉の山賊焼き
week,,,,,,,,,,,,,,,,,,,,,
月,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
木,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,0
水,0,0,0,0,0,1,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
火,0,1,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,0
金,0,0,0,0,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0


# カラムのメニュー名に重複がないかどうか確認

In [6]:
'''ピボットテーブルのカラムをリストに格納'''
name_list = pivot_df.T.index.tolist()
name_list[0:3]

['いか天ぷら', 'かじきの甘辛煮', 'きのこソースハンバーグ']

In [7]:
'''リスト内に重複した要素があるかを判定する関数'''
# 参考URL：https://note.nkmk.me/python-list-duplicate-check/
def is_unique(list_type_obj):
    '''
    list_type_obj:重複判定したいリスト型の変数を指定
    
    set型は重複した要素を持たないデータ型で、コンストラクタset()にリストを渡すと、
    重複する値は無視されて一意な値のみが要素となるset型のオブジェクトを返す
    
    Trueが返ってきたらリスト内の要素に重複なし、Falseが返ってきたらリスト内の要素に重複あり
    '''
    return len(list_type_obj) == len(set(list_type_obj))

In [8]:
is_unique(name_list)

True

### →pivot_tableメソッドを使った場合、カラムの要素はユニークになることを確認

## BoW(メニュー名出現したかどうか(=0 or 1))

In [9]:
pivot_df2 = pd.pivot_table(reset_g_df, index='week', columns='name', values='count', aggfunc=len, fill_value=0)
pivot_df2

name,いか天ぷら,かじきの甘辛煮,きのこソースハンバーグ,さっくりメンチカツ,さわら焼味噌掛け,さんま辛味焼,たっぷりベーコンフライ,ひやしたぬきうどん・炊き込みご飯,ぶりレモンペッパー焼き,ますのマスタードソース,...,鶏の唐揚げ甘酢あん,鶏の塩から揚げ,鶏の天ぷら,鶏の照り焼きマスタード,鶏の照り焼きマヨ,鶏の親子煮,鶏チリソース,鶏肉とカシューナッツ炒め,鶏肉のカレー唐揚,鶏肉の山賊焼き
week,,,,,,,,,,,,,,,,,,,,,
月,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
木,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,0,0,0
水,0,0,0,0,0,1,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
火,0,1,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,0
金,0,0,0,0,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0


In [10]:
display(pivot_df['マーボ豆腐'], pivot_df2['マーボ豆腐'])

week
月    2
木    0
水    0
火    2
金    0
Name: マーボ豆腐, dtype: int64

week
月    1
木    0
水    0
火    1
金    0
Name: マーボ豆腐, dtype: int64